https://adventofcode.com/2022/day/22

In [3]:
import re
import logging

logging.basicConfig()
logger = logging.getLogger()

In [4]:
with open("data/22.txt")as fh:
    data = fh.read()

In [5]:
testdata = """\
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
"""

In [6]:
def load_data(data):
    boardstr, guidestr = data.split("\n\n")
    guide = [int(x) if x.isdigit() else x for x in re.findall(r"[RL]|\d+", guidestr)]
    board = {}
    edges = {1: {}, -1: {}, 0+1j: {}, 0-1j: {}}
    for minusy, line in enumerate(boardstr.splitlines(), 1):
        for x, c in enumerate(line, 1):
            if c in (".", "#"):
                y = -minusy
                board[complex(x, y)] = {".": 0, "#": 1}[c]
                edges[1][y] = max(edges[1].get(y, x), x)
                edges[-1][y] = min(edges[-1].get(y, x), x)
                edges[0+1j][x] = max(edges[0+1j].get(x, y), y)
                edges[0-1j][x] = min(edges[0-1j].get(x, y), y)
    extrema = {}
    for y, xmax in edges[1].items():
        xmin = edges[-1][y]
        right = complex(xmax, y)
        left = complex(xmin, y)
        extrema[(1, right)] = left
        extrema[(-1, left)] = right
    for x, ymax in edges[0+1j].items():
        ymin = edges[0-1j][x]
        top = complex(x, ymax)
        bottom = complex(x, ymin)
        extrema[(0+1j, top)] = bottom
        extrema[(0-1j, bottom)] = top

    return guide, board, extrema

In [7]:
def traverse(guide, board, extrema):
    starty = max(p.imag for p in board)
    startx = min(p.real for p in board if p.imag == starty)
    pos = complex(startx, starty)
    # print("start", pos)
    drxn = 1
    for step in guide:
        turn = {"R": 0-1j, "L": 0+1j}.get(step)
        if turn is not None:
            drxn *= turn
            # print(pos, drxn)
            continue
        for _ in range(step):
            nxt = pos + drxn
            try:
                tile = board[nxt]
            except KeyError:
                nxt = extrema[(drxn, pos)]
                tile = board[nxt]
            if tile:
                # hit the wall
                break
            else:
                pos = nxt
    password = -1000 * pos.imag + 4 * pos.real + {1: 0, 0-1j: 1, -1: 2, 0+1j: 3}[drxn]
    return pos, drxn, password

In [8]:
traverse(*load_data(testdata))

((8-6j), (1+0j), 6032.0)

In [9]:
%%time
traverse(*load_data(data))

CPU times: user 35.3 ms, sys: 0 ns, total: 35.3 ms
Wall time: 34.9 ms


((35-189j), (1+0j), 189140.0)

Part 2

<img src="22-cube.jpg" width="300" height="400"/>

In [75]:
def transition(face_point, from_drxn, to_drxn, side=50):
    xmin, xmax = 0, side-1
    ymin, ymax = 1-side, 0

    def up_up(p):
        return complex(p.real, ymin)

    def up_right(p):
        return complex(xmin, ymax - p.real)

    def up_down(p):
        return complex(xmax - p.real, ymax)

    def up_left(p):
        return complex(xmax, ymin + p.real)

    def right_right(p):
        return complex(xmin, p.imag)

    def right_down(p):
        return complex(p.imag - ymin, ymax)

    def right_left(p):
        return  complex(xmax, ymin - p.imag)

    def right_up(p):
        return complex(ymax - p.imag, ymin)

    def down_down(p):
        return complex(p.real, ymax)

    def down_left(p):
        return complex(xmax, -p.real)

    def down_up(p):
        return complex(xmax - p.real, ymin)

    def down_right(p):
        return complex(xmin, ymin + p.real)

    def left_left(p):
        return complex(xmax, p.imag)

    def left_up(p):
        return complex(p.imag - ymin, ymin)

    def left_right(p):
        return complex(xmin, ymin - p.imag)

    def left_down(p):
        return complex(ymax - p.imag, ymax)

    D = {
        (0+1j, 0+1j): up_up,
        (0+1j, 1): up_right,
        (0+1j, 0-1j): up_down,
        (0+1j, -1): up_left,

        (1, 1): right_right,
        (1, 0-1j): right_down,
        (1, -1): right_left,
        (1, 0+1j): right_up,

        (0-1j, 0-1j): down_down,
        (0-1j, -1): down_left,
        (0-1j, 0+1j): down_up,
        (0-1j, 1): down_right,

        (-1, -1): left_left,
        (-1, 0+1j): left_up,
        (-1, 1): left_right,
        (-1, 0-1j): left_down,
    }

    new_pos = D[(from_drxn, to_drxn)](face_point)
    return new_pos

In [73]:
class Cube:
    side = 4
    face_offsets = {
        1: 0,
        2: complex(-2 * side, -side),
        3: complex(-side, -side),
        4: complex(0, -side),
        5: complex(0, -2 * side),
        6: complex(side, -2 * side)
    }
    neighbors = {
        1: {
            0+1j: 2,
            1: 6,
            0-1j: 4,
            -1: 3,
        },
        2: {
            0+1j: 1,
            1: 3,
            0-1j: 5,
            -1: 6,
        },
        3: {
            0+1j: 1,
            1: 4,
            0-1j: 5,
            -1: 2,
        },
        4: {
            0+1j: 1,
            1: 6,
            0-1j: 5,
            -1: 3,
        },
        5: {
            0+1j: 4,
            1: 6,
            0-1j: 2,
            -1: 3,
        },
        6: {
            0+1j: 4,
            1: 1,
            0-1j: 2,
            -1: 5,
        },
    }

    def __init__(self, board):
        self.board = board
        self.set_transition_drxn()
        self.set_faces()

    def set_transition_drxn(self):
        transition_drxn = {}
        for face_id, nabes in self.neighbors.items():
            transition_drxn[face_id] = {v: -k for (k, v) in nabes.items()}
        self.transition_drxn = transition_drxn

    def set_faces(self):
        o_y = max(p.imag for p in self.board)
        o_x = min(p.real for p in self.board if p.imag == o_y)
        origin = self.origin = complex(o_x, o_y)
        s = self.side
        logger.debug("origin %s", origin)

        def extract_face(offset):
            face_origin = origin + offset
            logger.debug("offset %s, face_origin %s"), offset, face_origin
            keys = (complex(x, -y) for x in range(s) for y in range(s))
            return {k: self.board[k + face_origin] for k in keys}

        self.faces = {k: extract_face(v) for k, v in self.face_offsets.items()}

    def explore(self, guide):
        # L = []
        faces = self.faces
        face_id = 1
        face = faces[face_id]
        pos = 0
        drxn = 1

        for step in guide:
            turn = {"R": 0-1j, "L": 0+1j}.get(step)
            if turn is not None:
                drxn *= turn
                continue
            for _ in range(step):
                nxt = pos + drxn
                try:
                    nxt_val = face[nxt]
                except KeyError:
                    new_face_id, new_face, new_pos, new_drxn = self.next_face(pos, face_id, drxn)
                    if new_face_id is None:
                        break
                    else:
                        face_id, face, nxt, drxn =  new_face_id, new_face, new_pos, new_drxn
                        nxt_val = face[nxt]
                if nxt_val:
                    break
                pos = nxt

        board_pos = self.face_to_board_pos(face_id, pos)
        password = -1000 * board_pos.imag + 4 * board_pos.real + {1: 0, 0-1j: 1, -1: 2, 0+1j: 3}[drxn]
        return face_id, pos, board_pos, drxn, password

    def face_to_board_pos(self, face_id, p):
        return p + self.face_offsets[face_id] + self.origin

    def next_face(self, pos, face_id, drxn):
        new_face_id = self.neighbors[face_id][drxn]
        new_face = self.faces[new_face_id]
        new_drxn = self.transition_drxn[new_face_id][face_id]
        new_pos = transition(pos, drxn, new_drxn, side=self.side)
        try:
            new_val = new_face[new_pos]
        except:
            logger.exception("pos %s, face_id %s, drxn %s, new_pos %s, new_face_id %s, new_drxn %s", pos, face_id, drxn, new_pos, new_face_id, new_drxn)
            raise
        if new_val:  # hit the wall
            return None, None, None, None
        else:
            return new_face_id, new_face, new_pos, new_drxn


In [65]:
test_guide, test_board, test_extrema = load_data(testdata)

In [74]:
logger.setLevel("INFO")
c = Cube(test_board)
face_id, pos, board_pos, drxn, password = c.explore(test_guide)
print(face_id, pos, board_pos, drxn, password)

3 (2+0j) (7-5j) 1j 5031.0


In [72]:
class PuzzleCube(Cube):
    side = 50
    face_offsets = {
        1: 0,
        2: complex(side, 0),
        3: complex(0, -side),
        4: complex(-side, -2 * side),
        5: complex(0, -2 * side),
        6: complex(-side, -3 * side)
    }
    neighbors = {
        1: {
            0+1j: 6,
            1: 2,
            0-1j: 3,
            -1: 4,
        },
        2: {
            0+1j: 6,
            1: 5,
            0-1j: 3,
            -1: 1,
        },
        3: {
            0+1j: 1,
            1: 2,
            0-1j: 5,
            -1: 4,
        },
        4: {
            0+1j: 3,
            1: 5,
            0-1j: 6,
            -1: 1,
        },
        5: {
            0+1j: 3,
            1: 2,
            0-1j: 6,
            -1: 4,
        },
        6: {
            0+1j: 4,
            1: 5,
            0-1j: 2,
            -1: 1,
        },
    }


In [67]:
guide, board, extrema = load_data(data)

In [70]:
%%time
logger.setLevel("INFO")
c = PuzzleCube(board)
face_id, pos, board_pos, drxn, password = c.explore(guide)

print(face_id, pos, board_pos, drxn, password)

4 (14-14j) (15-115j) (-0+1j) 115063.0
CPU times: user 13.8 ms, sys: 123 µs, total: 13.9 ms
Wall time: 13.3 ms
